In [1]:
import numpy as np
np.random.seed(1337) # for reproducibility

In [2]:
from tensorflow import keras
import pandas as pd

from src.dataframe import importDfPickle

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D, MaxPooling2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.python.keras.optimizers import Adadelta, Adam
from keras.utils import to_categorical

classifier = Sequential()

Using TensorFlow backend.


In [3]:
df=importDfPickle("./output/df_30mil.pkl")

In [4]:
df.shape

(30000, 3)

In [6]:
#convertir la columna en una numpy que tenga la dimension filas, 50,50,3
X=np.asarray(list(df['image']))
y=np.asarray(df['label'])

In [7]:
X.shape

(30000, 50, 50, 3)

In [8]:
def data_summary(X_train, y_train, X_test, y_test):
    """Summarize current state of dataset"""
    print('Train images shape:', X_train.shape)
    print('Train labels shape:', y_train.shape)
    print('Test images shape:', X_test.shape)
    print('Test labels shape:', y_test.shape)

In [9]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [10]:
data_summary(X_train, y_train, X_test, y_test)

Train images shape: (24000, 50, 50, 3)
Train labels shape: (24000,)
Test images shape: (6000, 50, 50, 3)
Test labels shape: (6000,)


In [11]:
"""import matplotlib.pyplot as plt
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(X_train[i], cmap=plt.cm.binary)
    plt.xlabel(y_train[i])
plt.show()"""

'import matplotlib.pyplot as plt\nplt.figure(figsize=(10,10))\nfor i in range(25):\n    plt.subplot(5,5,i+1)\n    plt.xticks([])\n    plt.yticks([])\n    plt.grid(False)\n    plt.imshow(X_train[i], cmap=plt.cm.binary)\n    plt.xlabel(y_train[i])\nplt.show()'

In [12]:
# Ask keras which format to use depending on used backend and arrange data as expected
"""if K.image_data_format() == 'channels_first':
    X_train = x_train.reshape(X_train.shape[0], 3, 50, 50)
    X_test = x_test.reshape(X_test.shape[0], 3, 50, 50)
    input_shape = (3, 50, 50)
else:
    X_train = X_train.reshape(X_train.shape[0], 50, 50, 3)
    X_test = X_test.reshape(X_test.shape[0], 50, 50, 3)
    input_shape = (50, 50, 3)"""

"""# Incoming data is in uint8. Cast the input data images to be floats in range [0.0-1.0]  
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255"""

""" 
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')
# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, 2)
y_test = keras.utils.to_categorical(y_test, 2)"""

" \nprint('x_train shape:', X_train.shape)\nprint(X_train.shape[0], 'train samples')\nprint(X_test.shape[0], 'test samples')\n# convert class vectors to binary class matrices\ny_train = keras.utils.to_categorical(y_train, 2)\ny_test = keras.utils.to_categorical(y_test, 2)"

In [13]:
## This is the neural network proposed architecture
model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=(50,50,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))
model.compile(loss=keras.losses.sparse_categorical_crossentropy, #sparse_categorical_crossentropy
              optimizer='adadelta',
              metrics=['accuracy'])

In [14]:
# Fit the NN
batch_size = 50
epochs = 14

model.fit(X_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, y_test))

Train on 24000 samples, validate on 6000 samples
Epoch 1/14
24000/24000 [==============================] - 191s 8ms/sample - loss: 23.2442 - accuracy: 0.5002 - val_loss: 2.8736 - val_accuracy: 0.5478
Epoch 2/14
24000/24000 [==============================] - 175s 7ms/sample - loss: 12.5200 - accuracy: 0.5208 - val_loss: 1.2441 - val_accuracy: 0.6585
Epoch 3/14
24000/24000 [==============================] - 179s 7ms/sample - loss: 6.5922 - accuracy: 0.5393 - val_loss: 0.7446 - val_accuracy: 0.6433
Epoch 4/14
24000/24000 [==============================] - 185s 8ms/sample - loss: 3.3853 - accuracy: 0.5472 - val_loss: 0.6507 - val_accuracy: 0.6387
Epoch 5/14
24000/24000 [==============================] - 205s 9ms/sample - loss: 1.8235 - accuracy: 0.5429 - val_loss: 0.6772 - val_accuracy: 0.5775
Epoch 6/14
24000/24000 [==============================] - 218s 9ms/sample - loss: 1.1541 - accuracy: 0.5427 - val_loss: 0.6852 - val_accuracy: 0.6562
Epoch 7/14
24000/24000 [=========================

In [15]:
# Evaluate the model with test data
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.670122140566508
Test accuracy: 0.6781667


In [16]:
import json

name='Seq 50,14 acu735'

model_json = model.to_json()
with open(name+'.json', "w") as json_file:
    json.dump(model_json, json_file)

model.save_weights(name+'.h5')

### Confusion matrix

In [16]:
from sklearn.metrics import confusion_matrix, make_scorer, accuracy_score

In [ ]:
#X_test = X_test.astype('float32')
#y_pred = model.predict(X_test)

y_pred = [int(round(model.predict(np.expand_dims(e,axis=0))[0][0])) for e in X_test.astype('float32')]
y_true = [int(e[0]) for e in y_test]

In [25]:
plt.clf()
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
classNames = ['Tumor','Not tumor']
plt.title('Tumor or Not Tumor Confusion Matrix - Test Data')
plt.ylabel('True label')
plt.xlabel('Predicted label')
tick_marks = np.arange(len(classNames))
plt.xticks(tick_marks, classNames, rotation=45)
plt.yticks(tick_marks, classNames)
s = [['TN','FP'], ['FN', 'TP']]
for i in range(2):
    for j in range(2):
        plt.text(j,i, str(s[i][j])+" = "+str(cm[i][j]))
plt.savefig('../output/cm')
plt.show()

TypeError: Singleton array 0 cannot be considered a valid collection.

In [ ]:
#mirar lo del tumor cerebral para confusion matrix

#### Predicciones

In [12]:
from keras.models import model_from_json
import json

# load json and create model
with open('Seq 50,14 acu735.json', 'r') as jfile:
    loaded_model = model_from_json(json.load(jfile))

# load weights into new model
loaded_model.load_weights("Seq 50,14 acu735.h5")
print("Loaded model from disk")

loaded_model.save('Seq 50,14 acu735.hdf5')
loaded_model=load_model('Seq 50,14 acu735.hdf5')

KeyError: 0

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
from PIL import Image

img = X_test[30]
pred = model.predict(np.expand_dims(img,axis=0))[0]
print("Probs -> Benign:{0:.5f} Malignant:{1:.5f}".format(pred[0],pred[1]))
#plt.imshow()



In [13]:
print(keras.__version__)

2.2.4-tf
